# Sample Locations
----
To install this package with conda run one of the following:
conda install -c esri arcpy
conda install -c esri/label/prerelease arcpy


In [32]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
import os
 

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [36]:
#load the csv
spoints_df = "data/Platform_Location.csv"
spoints_df = pd.read_csv(spoints_df)
#spoints_df.head()

FileNotFoundError: [Errno 2] File data/Platform_Location.csv does not exist: 'data/Platform_Location.csv'

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [82]:
#config gmaps
gmaps.configure(api_key=g_key)

#store the lat and long
locations = weather_df[["Latitude", "Longitude"]].astype(float)

#store humidity in humidity
humidity_rate = weather_df["Humidity"].astype(float)

In [92]:
# create a poverty Heatmap layer

fig = gmaps.figure(center=(48.0, 0.0), zoom_level=1)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, dissipating=False, max_intensity=100, point_radius = 2, opacity=1)
 
fig.add_layer(heat_layer)
plt.savefig("../Images/Vacation_Heat_Map.png")

fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [84]:
#ideal temp, wind speed, and cloudiness
ideal_df = weather_df.loc[(weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80) & (weather_df["Cloudiness"] == 0), :]
ideal_df = ideal_df.dropna(how='any')
ideal_df.reset_index(inplace=True)
del ideal_df['index']
ideal_df.head(5)



,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
0,8,Mabaruma,8.20,-59.78,77.34,89,0,3.47,1599964581,GY
1,45,Acaraú,-2.89,-40.12,77.54,82,0,18.05,1599964594,BR
2,51,Geraldton,-28.77,114.60,71.60,43,0,17.22,1599964596,AU
3,65,Carnarvon,-24.87,113.63,78.80,29,0,17.22,1599964601,AU
4,115,Aksu,41.12,80.26,71.15,38,0,3.47,1599964620,CN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [85]:
hotel_df = pd.DataFrame()

hotel_df['City'] = ideal_df['City']
hotel_df['Country'] = ideal_df['Country']
hotel_df['Latitude'] = ideal_df['Latitude']
hotel_df['Longitude'] = ideal_df['Longitude']
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Latitude,Longitude,Hotel Name
0,Mabaruma,GY,8.20,-59.78,
1,Acaraú,BR,-2.89,-40.12,
2,Geraldton,AU,-28.77,114.60,
3,Carnarvon,AU,-24.87,113.63,
4,Aksu,CN,41.12,80.26,


In [86]:
# create hotel file
hotel_df = ideal_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# parameters
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # latitudes and longitudes from df
    Latitude = row["Latitude"]
    Longitude = row["Longitude"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{Latitude},{Longitude}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    #print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

Missing field/result... skipping.


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country,Hotel Name
0,8,Mabaruma,8.20,-59.78,77.34,89,0,3.47,1599964581,GY,
1,45,Acaraú,-2.89,-40.12,77.54,82,0,18.05,1599964594,BR,La Villa Praia
2,51,Geraldton,-28.77,114.60,71.60,43,0,17.22,1599964596,AU,The Gerald Apartment Hotel
3,65,Carnarvon,-24.87,113.63,78.80,29,0,17.22,1599964601,AU,Carnarvon Motel
4,115,Aksu,41.12,80.26,71.15,38,0,3.47,1599964620,CN,华龙酒店
5,135,Kiryat Gat,31.61,34.76,73.24,9,0,0.40,1599964628,IL,Cramim Resort & Spa
6,211,Caravelas,-17.71,-39.25,70.97,86,0,6.53,1599964658,BR,Residencial Lagoa da Praia
7,244,Coruripe,-10.13,-36.18,73.15,79,0,11.56,1599964672,BR,Kenoa
8,262,Diamantino,-14.41,-56.45,79.77,25,0,6.42,1599964678,BR,Hotel Villa Dos parecis
9,286,São João da Barra,-21.64,-41.05,72.90,84,0,25.01,1599964686,BR,Refúgio Nômade Hostel


In [87]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Latitude", "Longitude"]]

In [91]:

# Marker added to mark hotels
markers = gmaps.marker_layer(locations2,
    info_box_content=[f"Hotel: {rate}" for rate in hotel_info])

fig.add_layer(markers)
fig.add_layer(heat_layer)

plt.savefig("../Images/VacationHotel_Heat_Map.png")
# Display Map
fig        



Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>